# Typing Profiling: CyberLab Human Model → Cowrie Scoring

This notebook builds a **human interaction model** from CyberLab, then applies it to Cowrie honeypot sessions to estimate how **human-like** each session’s command timing appears.

We present three levels of inference:

1. **Simple PDF Baseline (Quantile Rule)**
   Scores sessions under the CyberLab human probability density function (PDF) and labels them using **CyberLab-derived log-likelihood quantiles**.

2. **Primary: Human-Likeness Tail Test (Mahalanobis χ²)**
   Uses the global CyberLab Gaussian and converts distance-to-human into a **tail probability** `p_human_tail` (smaller ⇒ less human-like).

3. **Secondary: Human-vs-Background Posterior (Gated Mixture)**
   Adds a background (bot-like) density model (GMM) and computes a **posterior** `p(human | x)` while still keeping CyberLab as the human anchor.

Outputs are written to `./output/` as compact, reusable CSVs.


In [2]:
import os
import glob
import numpy as np
import pandas as pd

from pathlib import Path
from scipy.stats import chi2
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.mixture import GaussianMixture

OUT_DIR = Path("output")
OUT_DIR.mkdir(exist_ok=True, parents=True)

# Inputs
CYBERLAB_SESS_PATH = "output/cyberlab_clustering_features.csv"
COWRIE_LINES_PATH  = "../../fi_fs/data/processed/Cowrie_Merged_Geo_Enriched_lines.csv"

# Outputs
HUMAN_MODEL_NPZ    = OUT_DIR / "human_gaussian_densities.npz"
COWRIE_DENSITY_CSV = OUT_DIR / "cowrie_session_density_features.csv"
COWRIE_SCORED_CSV  = OUT_DIR / "cowrie_session_typing_scored.csv"
COWRIE_A_CSV       = OUT_DIR / "cowrie_session_typing_humanlikeness_tailtest.csv"
COWRIE_B_CSV       = OUT_DIR / "cowrie_session_typing_posterior_gatedmixture.csv"

print("Paths:")
print("  CYBERLAB_SESS_PATH:", CYBERLAB_SESS_PATH)
print("  COWRIE_LINES_PATH :", COWRIE_LINES_PATH)
print("  HUMAN_MODEL_NPZ   :", HUMAN_MODEL_NPZ)
print("  COWRIE_DENSITY_CSV:", COWRIE_DENSITY_CSV)
print("  COWRIE_SCORED_CSV :", COWRIE_SCORED_CSV)
print("  COWRIE_A_CSV      :", COWRIE_A_CSV)
print("  COWRIE_B_CSV      :", COWRIE_B_CSV)


Paths:
  CYBERLAB_SESS_PATH: output/cyberlab_clustering_features.csv
  COWRIE_LINES_PATH : ../../fi_fs/data/processed/Cowrie_Merged_Geo_Enriched_lines.csv
  HUMAN_MODEL_NPZ   : output/human_gaussian_densities.npz
  COWRIE_DENSITY_CSV: output/cowrie_session_density_features.csv
  COWRIE_SCORED_CSV : output/cowrie_session_typing_scored.csv
  COWRIE_A_CSV      : output/cowrie_session_typing_humanlikeness_tailtest.csv
  COWRIE_B_CSV      : output/cowrie_session_typing_posterior_gatedmixture.csv


In [4]:
# Tukey upper fence for honeypot outlier removal
def tukey_upper_fence_pos(x: pd.Series) -> float:
    s = x[(x > 0) & np.isfinite(x)]
    if len(s) < 4:
        return np.inf
    q1, q3 = np.percentile(s, [25, 75])
    iqr = q3 - q1
    return np.inf if iqr <= 0 else (q3 + 1.5 * iqr)

# Gaussian fit utility
def fit_gaussian(df: pd.DataFrame, cols, reg=1e-6, min_samples=5):
    Z = df[cols].dropna().to_numpy()
    n, d = Z.shape
    if n < min_samples:
        raise ValueError(f"Need >= {min_samples} samples, got {n}.")
    mu = Z.mean(axis=0)
    Sigma = np.atleast_2d(np.cov(Z, rowvar=False)) + reg * np.eye(d)
    return mu, Sigma

def mvn_logpdf(Z, mu, Sigma):
    Z = np.atleast_2d(Z)
    mu = np.asarray(mu)
    Sigma = np.asarray(Sigma)
    d = Z.shape[1]
    try:
        L = np.linalg.cholesky(Sigma)
    except np.linalg.LinAlgError:
        L = np.linalg.cholesky(Sigma + 1e-5 * np.eye(d))
    diff = (Z - mu)
    sol = np.linalg.solve(L, diff.T)
    maha = np.sum(sol**2, axis=0)
    log_det = 2.0 * np.sum(np.log(np.diag(L)))
    log_norm = 0.5 * (d * np.log(2.0 * np.pi) + log_det)
    return -log_norm - 0.5 * maha

def logsumexp(a, axis=1, keepdims=True):
    a = np.asarray(a)
    m = np.max(a, axis=axis, keepdims=True)
    out = np.log(np.sum(np.exp(a - m), axis=axis, keepdims=True)) + m
    return out if keepdims else out.squeeze()

# Working with three labels currently
# human_like = positively human (high score)
# uncertain = close to human threshold, not super confident
# non_human_like = negatively human (low score)
def three_way_label(values, thr_hi, thr_lo, hi="human_like", mid="uncertain", lo="non_human_like"):
    # values high = more human-like
    return np.where(values >= thr_hi, hi, np.where(values >= thr_lo, mid, lo))

def top_bottom(df, score_col, n=3, min_pairs=None):
    x = df.copy()
    if min_pairs is not None:
        x = x[x["n_pairs"] >= int(min_pairs)].copy()
    top = x.sort_values(score_col, ascending=False).head(n).copy()
    bot = x.sort_values(score_col, ascending=True).head(n).copy()
    top.insert(0, "rank_group", "most_human_like")
    bot.insert(0, "rank_group", "most_non_human_like")
    return pd.concat([top, bot], ignore_index=True)


## Part 1 - Build the CyberLab human model (anchor distribution)

We fit a global multivariate Gaussian in a **log feature space** derived from:
- `mean_spc`, `std_spc`, `median_dt`, `median_ts`

This defines the CyberLab **human interaction PDF**.


In [9]:
df_sess = pd.read_csv(CYBERLAB_SESS_PATH)
print("CyberLab sessions loaded:", len(df_sess))

# Match prior convention: drop SQL injection sessions
df_clust = df_sess[df_sess.get("sample", "") != "SQL injection"].copy()

# Reproduce clustering (k=5) so cluster affinities are available later
cluster_features = [c for c in [
    "median_ts","median_dt","n_pairs","median_len","mean_spc","std_spc","avg_time_per_char"
] if c in df_clust.columns]

if not cluster_features:
    raise ValueError("No clustering features found in CyberLab CSV.")

X = df_clust[cluster_features].copy()
for c in [c for c in ["n_pairs","median_dt","median_ts","mean_spc","std_spc","avg_time_per_char"] if c in X.columns]:
    X[c] = np.log1p(X[c])

X_scaled = StandardScaler().fit_transform(X)
df_clust["cluster"] = KMeans(n_clusters=5, random_state=42, n_init=10).fit_predict(X_scaled)

print("CyberLab cluster counts:")
print(df_clust["cluster"].value_counts().sort_index())

# Density feature space (log domain)
eps = 1e-6
required = ["mean_spc","std_spc","median_dt","median_ts"]
missing = [c for c in required if c not in df_clust.columns]
if missing:
    raise ValueError(f"CyberLab missing required columns for density model: {missing}")

df_clust["log_mean_spc"]  = np.log(df_clust["mean_spc"]  + eps)
df_clust["log_std_spc"]   = np.log(df_clust["std_spc"]   + eps)
df_clust["log_median_dt"] = np.log(df_clust["median_dt"] + eps)
df_clust["log_median_ts"] = np.log(df_clust["median_ts"] + eps)

density_cols = ["log_mean_spc","log_std_spc","log_median_dt","log_median_ts"]
df_cy = df_clust.dropna(subset=density_cols).copy()

print("CyberLab rows with density features:", len(df_cy), "/", len(df_clust))
df_cy[density_cols].describe().T.round(3)


CyberLab sessions loaded: 166
CyberLab cluster counts:
cluster
0    53
1    62
2     2
3     2
4    43
Name: count, dtype: int64
CyberLab rows with density features: 162 / 162


,count,mean,std,min,25%,50%,75%,max
log_mean_spc,162.0,1.044,0.647,-0.598,0.560,1.066,1.544,2.288
log_std_spc,162.0,1.558,0.737,-0.165,1.037,1.610,2.175,3.246
log_median_dt,162.0,2.607,1.184,-6.907,2.079,2.697,3.324,5.342
log_median_ts,162.0,-0.038,1.157,-2.297,-0.539,-0.138,0.343,9.649


In [12]:
mu_h, Sigma_h = fit_gaussian(df_cy, density_cols, reg=1e-6, min_samples=5)
print("Fitted CyberLab global human Gaussian.")

# Per-cluster Gaussians (human “types”)
cluster_gaussians = {}
cluster_ids = []
for k, df_k in df_cy.groupby("cluster"):
    try:
        mu_k, Sigma_k = fit_gaussian(df_k, density_cols, reg=1e-6, min_samples=2)
        cluster_gaussians[int(k)] = (mu_k, Sigma_k, len(df_k))
        cluster_ids.append(int(k))
    except ValueError:
        pass

cluster_ids = sorted(cluster_ids)
priors = (
    df_cy[df_cy["cluster"].isin(cluster_ids)]["cluster"]
    .value_counts(normalize=True).sort_index().reindex(cluster_ids).to_numpy()
)
mus = np.stack([cluster_gaussians[cid][0] for cid in cluster_ids])
Sigmas = np.stack([cluster_gaussians[cid][1] for cid in cluster_ids])

np.savez(
    HUMAN_MODEL_NPZ,
    feature_cols=np.array(density_cols),
    mu_global=mu_h,
    Sigma_global=Sigma_h,
    cluster_ids=np.array(cluster_ids),
    cluster_priors=priors,
    mus=mus,
    Sigmas=Sigmas,
)

print("Saved model:", HUMAN_MODEL_NPZ)
print("cluster_ids:", cluster_ids)
# Gaussian mixture priors represent the proportion of CyberLab sessions in each cluster
print("priors:", np.round(priors, 3))


Fitted CyberLab global human Gaussian.
Saved model: output/human_gaussian_densities.npz
cluster_ids: [0, 1, 2, 3, 4]
priors: [0.327 0.383 0.012 0.012 0.265]


## Part 2 - Build Cowrie session density features (from line-level logs)

Cowrie is line-level. We reconstruct per-session interaction timing by:
1) sorting by `session, timestamp`,
2) taking inter-command diffs (`time_diff`),
3) applying **within-session Tukey** to remove long breaks,
4) deriving seconds-per-char (SPC) and typing speed (CPS),
5) aggregating to a session-level table.

We then create the same **log density features** required by the CyberLab model.


In [13]:
df_lines = pd.read_csv(COWRIE_LINES_PATH)
df_lines["timestamp"] = pd.to_datetime(df_lines["timestamp"], utc=True, errors="coerce")
df_lines = df_lines.dropna(subset=["timestamp","session"]).copy()
df_lines = df_lines.sort_values(["session","timestamp"])

df_lines["time_diff"] = df_lines.groupby("session")["timestamp"].diff().dt.total_seconds()
df_lines["cmd_length"] = df_lines["message"].astype(str).str.len()

df_pairs = df_lines[(df_lines["time_diff"] > 0) & (df_lines["cmd_length"] > 0)].copy()
print("Pair rows before Tukey:", len(df_pairs))

df_pairs["upper_fence"] = df_pairs.groupby("session")["time_diff"].transform(tukey_upper_fence_pos)
df_pairs = df_pairs[df_pairs["time_diff"] <= df_pairs["upper_fence"]].copy()
print("Pair rows after Tukey :", len(df_pairs))

df_pairs["seconds_per_char"] = df_pairs["time_diff"] / df_pairs["cmd_length"]
df_pairs["typing_speed"]     = df_pairs["cmd_length"] / df_pairs["time_diff"]

# Session aggregation
MIN_PAIRS = 3
sess = (
    df_pairs.groupby("session", as_index=False)
    .agg(
        n_pairs=("time_diff","size"),
        mean_spc=("seconds_per_char","mean"),
        std_spc=("seconds_per_char","std"),
        median_dt=("time_diff","median"),
        median_ts=("typing_speed","median"),
    )
)
sess = sess[sess["n_pairs"] >= MIN_PAIRS].copy()
print(f"Sessions with >= {MIN_PAIRS} pairs:", len(sess))

# Density features
EPS = 1e-9
sess["log_mean_spc"]  = np.log(sess["mean_spc"].clip(lower=EPS))
sess["log_std_spc"]   = np.log(sess["std_spc"].fillna(sess["std_spc"].median()).clip(lower=EPS))
sess["log_median_dt"] = np.log(sess["median_dt"].clip(lower=EPS))
sess["log_median_ts"] = np.log(sess["median_ts"].clip(lower=EPS))

sess = sess.dropna(subset=density_cols).copy()

# Lightweight extra diagnostics
extra = (
    df_pairs.groupby("session")
    .agg(
        paste_ratio=("time_diff", lambda s: (s < 0.1).mean()),
        mean_dt=("time_diff","mean"),
        std_dt=("time_diff","std"),
    )
    .reset_index()
)
extra["dt_cv"] = extra["std_dt"] / extra["mean_dt"]
sess = sess.merge(extra[["session","paste_ratio","dt_cv"]], on="session", how="left")

sess.to_csv(COWRIE_DENSITY_CSV, index=False)
print("Saved:", COWRIE_DENSITY_CSV)

sess.head(5)


Pair rows before Tukey: 295901
Pair rows after Tukey : 270514
Sessions with >= 3 pairs: 49303
Saved: output/cowrie_session_density_features.csv


,session,n_pairs,mean_spc,std_spc,median_dt,median_ts,log_mean_spc,log_std_spc,log_median_dt,log_median_ts,paste_ratio,dt_cv
0,00031aeff1a6,3,0.000181,0.000105,0.001465,4191.114837,-8.619259,-9.158441,-6.525900,8.340722,1.000000,0.671601
1,0003e7887230,3,0.000192,0.000113,0.001505,3986.710963,-8.555930,-9.086703,-6.498962,8.290722,1.000000,0.661817
2,0003f10a2103,3,0.001741,0.002250,0.003836,1564.129301,-6.353453,-6.096984,-5.563325,7.355085,0.666667,1.685798
3,0004025aa9c2,11,0.000126,0.000078,0.012148,12347.711557,-8.978872,-9.459189,-4.410591,9.421226,1.000000,0.484879
4,000cd6e3c127,3,0.000338,0.000160,0.001457,4035.512510,-7.993555,-8.740878,-6.531376,8.302889,1.000000,1.540924
